In [12]:
from flask import Flask, jsonify, request, render_template 
import sqlite3
import json
import random


app = Flask(__name__) 

@app.route('/test',methods=['PUT'])
def test():
    if request.method == 'PUT':
        user = request.get_json(force=True)
        print(user)
        print('success')
    return 'success'
if __name__ == '__main__': 
    app.run(debug = False) 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


127.0.0.1 - - [20/Mar/2021 11:49:24] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2021 11:49:24] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2021 11:49:29] "GET /put HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2021 11:49:38] "GET /test HTTP/1.1" 405 -


In [1]:
#video to audio

import moviepy.editor as mp 
  
# Insert Local Video File Path  
clip = mp.VideoFileClip("force.mp4") 
  
# Insert Local Audio File Path 
clip.audio.write_audiofile("audio.wav")

chunk:   1%|▏         | 417/28931 [00:00<00:06, 4167.35it/s, now=None]

MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [8]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

sound = AudioSegment.from_file("audio.wav", format="wav")
chunks = split_on_silence(
    sound,

    # split on silences longer than 1000ms (1 sec)
    min_silence_len=500,

    # anything under -16 dBFS is considered silence
    silence_thresh=-40,
    keep_silence=200
)

# now recombine the chunks so that the parts are at least 90 sec long
target_length = 300 * 1000
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)
for i, chunk in enumerate(output_chunks):
    out_file = "audio_chunks/chunk{0}.wav".format(i)
    #print("exporting", out_file)
    chunk.export(out_file, format="wav")
    count = i

In [9]:
#pytorch based speech to text

import torch
import zipfile
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',model='silero_stt',language='en',device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# download a single file, any format compatible with TorchAudio (soundfile backend)
#torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               #dst ='speech_orig.wav', progress=True)
text =[]
b = ''
for i in range(len(output_chunks)):
    
    test_files = glob('audio_chunks/chunk{}.wav'.format(i))
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]),
                                device=device)

    output = model(input)
    for example in output:
        b+=decoder(example.cpu())
        #print(decoder(example.cpu()))
    text.append(b)
print(text)

Using cache found in /home/kush/.cache/torch/hub/snakers4_silero-models_master


["high friends in the last chapter we learnt all about motion and the different physics ters like distance displacement speed and veloity but what causes motion for example if i take this ball what do i need to do to give motion to the ball that's right i need to give it a push so i need to apply a force on the ball to give it motion and what is need ed to stop the motion of the ball again a force similarly if you consider the motion of car 's the engine that applies the force on the cart causing it to move and the braks apply a force to stop the car and forces going to be the topic of this video we 're going to learn all about force the different types of force and the effects of force i'm going to make the contcepts really easy for you and after you watch this video to try solving the quizs and the top three questions on this topic links are given below the video 'm sure you know what forces but in physics how can we define force there's a very simple definition in physics forces def